<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong>
        Importando bibliotecas
    </strong>
</p>

In [35]:
import pandas as pd
import xlwings as xw

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        Lendo base de dados
    </strong>
</p>

In [36]:
def ler_arquivo(nome_arquivo): 
    try: 
        df = pd.read_excel(f'dados/base_entregas_brutas.xlsx')
        return df
    except FileNotFoundError as e:
        print(e)

# try: 
#     df = pd.read_excel(f'dados/base_entregas_brutas.xlsx')
# except FileNotFoundError as e:
#     print(e)


def salvar_arquivo (nome_arquivo, df_tratado=''):
    df_tratado.to_excel(f'dados/{nome_arquivo}.xlsx',index=False)

In [37]:
df = ler_arquivo('base_entregas_brutas')

<p style="font-size: 1.1rem; color: #9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        Informações gerais
    </strong>
</p>

In [38]:
# df.shape
df.info()
# df.sample(5, random_state=42)
# df.sample(frac=0.1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_entrega         5200 non-null   int64         
 1   filial             4454 non-null   object        
 2   cliente            4283 non-null   object        
 3   status             4615 non-null   object        
 4   descricao_entrega  4637 non-null   object        
 5   tipo_servico       4095 non-null   object        
 6   valor_frete        5200 non-null   float64       
 7   data_postagem      5200 non-null   datetime64[ns]
 8   prazo_previsto     4330 non-null   float64       
 9   dias_atraso        4479 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 406.4+ KB


<p style="font-size: 1.1rem; color: #9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        Tratamento - Dados ausentes        
    </strong>
</p>

<p style="font-size: 1.1rem; color: #00d9ff; font-family: monospace ">
    <strong>
        Por segurança, criamos uma copia do DF       
    </strong>
</p>

In [39]:
df_tratado = df.copy()

<p style="font-size: 1.1rem; color: #00d9ff; font-family: monospace ">
    <strong>
        Filial 
    </strong>
</p>

In [40]:
df_tratado.loc[:,'filial'] = df_tratado['filial'].fillna('Não informada')

# df_tratado['filial'].sample(10)

<p style="font-size: 1.1rem; color: #00d9ff; font-family: monospace ">
    <strong>
        Cliente 
    </strong>
</p>

In [41]:
df_tratado.loc[:,'cliente'] = df_tratado['cliente'].fillna('Não informado')

# df_tratado['cliente'].sample(10)

<p style="font-size: 1.1rem; color: #00d9ff; font-family: monospace ">
    <strong>
        Dias de atraso 
    </strong>
</p>

In [42]:
df_tratado.loc[:,'dias_atraso'] = df_tratado['dias_atraso'].fillna(0)

# df_tratado['dias_atraso'].sample(10)

<p style="font-size: 1.1rem; color: #00d9ff; font-family: monospace ">
    <strong>
        Prazo previsto 
    </strong>
</p>

In [43]:
df_tratado.loc[:,'prazo_previsto'] = df_tratado['prazo_previsto'].fillna(
    df_tratado['prazo_previsto'].median()
)

# df_tratado.head(10)

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        Tratamento - Duplicadas
    </strong>
</p>

<p style="font-size: 1.1rem; color: #00d9ff; font-family: monospace ">
    <strong>
        Verificando duplicadas com base no ID 
    </strong>
</p>

In [44]:
df_tratado.duplicated(subset=['id_entrega']).sum()

2334

<p style="font-size: 1.1rem; color: #00d9ff; font-family: monospace ">
    <strong>
        Removendo duplicadas, mantendo a primeira ocorrencia
    </strong>
</p>

In [45]:
df_tratado.drop_duplicates(subset=['id_entrega'],inplace=True)

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        Tratamento - Tipagem correta para ID
    </strong>
</p>

In [46]:
df_tratado['id_entrega'] = df_tratado['id_entrega'].astype('Int64')
df_tratado['prazo_previsto'] = df_tratado['prazo_previsto'].astype('Int64')
df_tratado['dias_atraso'] = df_tratado['dias_atraso'].astype('Int64')


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        Tratamento - Padronização e Normalização
    </strong>
</p>

<p style="font-size: 1.1rem; color: #00d9ff; font-family: monospace ">
    <strong>
        Removendo espaços e alterando para caixa alta 
    </strong>
</p>

In [47]:
df_tratado['status'] = df_tratado['status'].str.strip().str.title()
df_tratado['tipo_servico'] = df_tratado['tipo_servico'].str.strip().str.title()

# df_tratado.head(10)

<p style="font-size: 1.1rem; color: #00d9ff; font-family: monospace ">
    <strong>
        Status 
    </strong>
</p>

In [48]:
mapa_status = {
    'Entregue' : 'Entregue',
    'Em Transito' : 'Em Trânsito',
    'Em Tranzito' : 'Em Trânsito',
    'Em Trânsito' : 'Em Trânsito',
    'Atrasado' : 'Atrasado',
    'Atrazado' : 'Atrasado'
}

df_tratado['status'] = df_tratado['status'].map(mapa_status)

# df_tratado['status'].head(10)

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/>
        Criando coluna calculada
    </strong>
</p>

In [49]:
df_tratado['data_prevista'] = (
    df_tratado['data_postagem']+
    pd.to_timedelta(df_tratado['prazo_previsto'], unit='D')
)

df_tratado[[
    'data_postagem',
    'prazo_previsto',
    'data_prevista'
]]
# df_tratado.head(10)

,data_postagem,prazo_previsto,data_prevista
0,2026-01-08 19:55:58.133,2,2026-01-10 19:55:58.133
1,2026-01-11 19:55:58.138,3,2026-01-14 19:55:58.138
2,2026-01-13 19:55:58.123,5,2026-01-18 19:55:58.123
3,2026-01-14 19:55:58.119,2,2026-01-16 19:55:58.119
4,2026-01-19 19:55:58.134,3,2026-01-22 19:55:58.134
...,...,...,...
5177,2026-01-29 19:55:58.143,5,2026-02-03 19:55:58.143
5184,2026-01-14 19:55:58.134,2,2026-01-16 19:55:58.134
5188,2026-01-08 19:55:58.141,3,2026-01-11 19:55:58.141
5190,2026-01-06 19:55:58.118,3,2026-01-09 19:55:58.118


In [50]:
df_tratado['atraso'] = (
    pd.to_datetime("today")-df_tratado['data_prevista']
).dt.days.clip(lower=0)

df_tratado.sample(10)

df_tratado[[
    'data_postagem',
    'prazo_previsto',
    'data_prevista',
    'atraso'
]]

,data_postagem,prazo_previsto,data_prevista,atraso
0,2026-01-08 19:55:58.133,2,2026-01-10 19:55:58.133,26
1,2026-01-11 19:55:58.138,3,2026-01-14 19:55:58.138,22
2,2026-01-13 19:55:58.123,5,2026-01-18 19:55:58.123,18
3,2026-01-14 19:55:58.119,2,2026-01-16 19:55:58.119,20
4,2026-01-19 19:55:58.134,3,2026-01-22 19:55:58.134,14
...,...,...,...,...
5177,2026-01-29 19:55:58.143,5,2026-02-03 19:55:58.143,2
5184,2026-01-14 19:55:58.134,2,2026-01-16 19:55:58.134,20
5188,2026-01-08 19:55:58.141,3,2026-01-11 19:55:58.141,25
5190,2026-01-06 19:55:58.118,3,2026-01-09 19:55:58.118,27


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Prazo previsto de acordo com a filial
    </strong>
</p>

In [51]:
df_tratado.loc[
    (df_tratado['cliente'] == 'Shoppe') &
    (df_tratado['filial'] == 'RJ'),
    'prazo_previsto'
] = 5

df_tratado.loc[
    (df_tratado['cliente'] == 'Amazon') &
    ((df_tratado['filial'].isin(["MG","SP"]))),
     "prazo_previsto"
] = 2

In [52]:
df_tratado

,id_entrega,filial,cliente,status,descricao_entrega,tipo_servico,valor_frete,data_postagem,prazo_previsto,dias_atraso,data_prevista,atraso
0,2870,PR,Amazon,Entregue,Entrega realizada no prazo,Normal,274.75,2026-01-08 19:55:58.133,2,0,2026-01-10 19:55:58.133,26
1,930,RJ,Mercado Livre,Entregue,Entrega reagendada,Normal,81.90,2026-01-11 19:55:58.138,3,0,2026-01-14 19:55:58.138,22
2,1934,MG,Amazon,Entregue,NaN,NaN,291.76,2026-01-13 19:55:58.123,2,3,2026-01-18 19:55:58.123,18
3,940,MG,Amazon,Atrasado,Entrega reagendada,Econômico,40.89,2026-01-14 19:55:58.119,2,0,2026-01-16 19:55:58.119,20
4,1969,SC,Mercado Livre,Em Trânsito,Entrega com atraso por chuva,NaN,229.75,2026-01-19 19:55:58.134,3,0,2026-01-22 19:55:58.134,14
...,...,...,...,...,...,...,...,...,...,...,...,...
5177,65,PR,Amazon,Atrasado,Extravio temporário,Normal,251.47,2026-01-29 19:55:58.143,5,2,2026-02-03 19:55:58.143,2
5184,1056,RJ,Magazine Luiza,Atrasado,Cliente ausente no local,Normal,320.94,2026-01-14 19:55:58.134,2,0,2026-01-16 19:55:58.134,20
5188,291,SP,Americanas,Em Trânsito,Entrega realizada no prazo,Normal,155.01,2026-01-08 19:55:58.141,3,0,2026-01-11 19:55:58.141,25
5190,1681,Não informada,Não informado,Em Trânsito,NaN,NaN,218.20,2026-01-06 19:55:58.118,3,0,2026-01-09 19:55:58.118,27


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Atualizando arquivo para executar fórmulas
    </strong>
</p>

In [ ]:
app = xw.App(visible=False)

wb = xw.Book('dados/base_entregas_tratadas.xlsx')

wb.app.calculate()

wb.save()

wb.close()

app.quit()

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/><br/><br/><br/><br/>
        Salvando arquivo tratado
    </strong>
</p>

In [54]:
salvar_arquivo('base_entregas_tratadas', df_tratado)